In [1]:
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
import tetrion.plots as plots
import tetrion.db
import tetrion.commands as cmd
import tetrion.options
import tetrion.clientcore
import seaborn as sns
from collections import defaultdict
import numpy as np
import warnings
import matplotlib.pyplot as plt
warnings.simplefilter("ignore")
%matplotlib inline 
symbol = pd.read_csv('/local/dist/tetrion/conf/conf/symbol.table', skiprows= 1, header = 2,  delim_whitespace=True)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
# import sys
# sys.path.append('../utils')  # Adjust the path to where the eventstudy module is located
# import eventstudy as es

In [2]:
import pandas as pd
import warnings
from tqdm import tqdm

warnings.filterwarnings("ignore")


In [111]:
warnings.filterwarnings("ignore", category=DeprecationWarning)
start_date = '2025-04-25'
end_date = '2025-05-06'
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

txf_books, txf_ticks = pd.DataFrame(), pd.DataFrame()

for date in tqdm(date_range, desc="Loading TXF data by day"):
    try:
        txf_book = cmd.book_printer_v2(
            'FUT_TAIFEX_TXF:202505',
            sdate=date,
            edate=date,
            depth=5,
            source='SRC_TAIFEX_RT_RD.ORIG_CPBQ.DEST_CPBQ'
        )
        txf_tick = cmd.tick_printer(
            'FUT_TAIFEX_TXF:202505',
            sdate=date,
            edate=date
        )

        txf_books = pd.concat([txf_books, txf_book], axis=0)
        txf_ticks = pd.concat([txf_ticks, txf_tick], axis=0)

    except Exception:
        continue


Loading TXF data by day:  17%|█▋        | 2/12 [00:58<04:20, 26.00s/it]WARNING: MYSQL_OPT_RECONNECT is deprecated and will be removed in a future version.
missing datafile SRC_TAIFEX.ORIG_CPBQ.DEST_CPBQ FUT_TAIFEX_TXF 20250427 file=/nfs/datafiles/2025/04/27/59709/datafile.944111726.gz
Loading TXF data by day:  75%|███████▌  | 9/12 [03:47<01:04, 21.35s/it]WARNING: MYSQL_OPT_RECONNECT is deprecated and will be removed in a future version.
missing datafile SRC_TAIFEX.ORIG_CPBQ.DEST_CPBQ FUT_TAIFEX_TXF 20250504 file=/nfs/datafiles/2025/05/04/59709/datafile.944111726.gz
Loading TXF data by day: 100%|██████████| 12/12 [05:17<00:00, 26.44s/it]


In [ ]:
# 確保 ex_time 是 datetime 格式
txf_ticks['ex_time'] = pd.to_datetime(txf_ticks['ex_time'])


txf_ticks = txf_ticks.rename(columns={'ex_time': 'time'}).set_index('time')

txf_ticks = txf_ticks.drop(columns=['time'], errors='ignore')  

txf_ticks.index.name = 'time'


In [6]:
merged_df = pd.merge(txf_ticks[['sz','px']], txf_books, on='time', how='outer', sort=True)
merged_df['sz'] = merged_df['sz'].fillna(0)

merged_df[txf_book.columns] = merged_df[txf_book.columns].ffill()

In [7]:
traded = merged_df[merged_df['sz'] != 0].copy()

In [8]:
traded[['ask_px','bid_px','px']]

,ask_px,bid_px,px
time,,,
2025-05-01 00:00:00.095,20089.0,20087.0,20088.0
2025-05-01 00:00:00.095,20089.0,20087.0,20088.0
2025-05-01 00:00:00.190,20089.0,20087.0,20089.0
2025-05-01 00:00:00.217,20088.0,20086.0,20088.0
2025-05-01 00:00:00.235,20088.0,20086.0,20088.0
...,...,...,...
2025-05-06 23:59:57.368,20312.0,20311.0,20312.0
2025-05-06 23:59:57.368,20312.0,20311.0,20312.0
2025-05-06 23:59:57.898,20312.0,20311.0,20311.0


In [ ]:

traded = merged_df[merged_df['sz'] != 0].copy()

# 定義穿價條件（避免 ask_px4 或 bid_px4 為 0）
up_cross = (traded['ask_px4'] != 0) & (traded['px'] > traded['ask_px4'])
down_cross = (traded['bid_px4'] != 0) & (traded['px'] < traded['bid_px4'])

# 建立穿價標籤
traded['trade_through'] = 'none'
traded.loc[up_cross, 'trade_through'] = 'upward'
traded.loc[down_cross, 'trade_through'] = 'downward'


# 只留下有穿價的成交
trade_through_df = traded[traded['trade_through'] != 'none']


In [10]:
trade_through_df

,sz,px,bid_sz4,bid_sz3,bid_sz2,bid_sz1,bid_sz,bid_px4,bid_px3,bid_px2,bid_px1,bid_px,mid_px,ask_px,ask_px1,ask_px2,ask_px3,ask_px4,ask_sz,ask_sz1,ask_sz2,ask_sz3,ask_sz4,trade_through
time,,,,,,,,,,,,,,,,,,,,,,,,
2025-05-01 01:44:24.460,5.0,20141.0,13.0,8.0,6.0,5.0,7.0,20129.0,20130.0,20131.0,20132.0,20133.0,20134.5,20136.0,20137.0,20138.0,20139.0,20140.0,5.0,6.0,18.0,11.0,11.0,upward
2025-05-01 01:44:24.460,8.0,20142.0,13.0,8.0,6.0,5.0,7.0,20129.0,20130.0,20131.0,20132.0,20133.0,20134.5,20136.0,20137.0,20138.0,20139.0,20140.0,5.0,6.0,18.0,11.0,11.0,upward
2025-05-01 01:44:24.460,4.0,20143.0,13.0,8.0,6.0,5.0,7.0,20129.0,20130.0,20131.0,20132.0,20133.0,20134.5,20136.0,20137.0,20138.0,20139.0,20140.0,5.0,6.0,18.0,11.0,11.0,upward
2025-05-01 01:44:24.460,20.0,20144.0,13.0,8.0,6.0,5.0,7.0,20129.0,20130.0,20131.0,20132.0,20133.0,20134.5,20136.0,20137.0,20138.0,20139.0,20140.0,5.0,6.0,18.0,11.0,11.0,upward
2025-05-01 01:44:24.460,4.0,20145.0,13.0,8.0,6.0,5.0,7.0,20129.0,20130.0,20131.0,20132.0,20133.0,20134.5,20136.0,20137.0,20138.0,20139.0,20140.0,5.0,6.0,18.0,11.0,11.0,upward
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-06 22:36:59.694,7.0,20297.0,6.0,8.0,9.0,8.0,1.0,20283.0,20284.0,20285.0,20286.0,20287.0,20288.0,20289.0,20290.0,20291.0,20292.0,20293.0,4.0,8.0,13.0,8.0,9.0,upward
2025-05-06 22:36:59.694,3.0,20298.0,6.0,8.0,9.0,8.0,1.0,20283.0,20284.0,20285.0,20286.0,20287.0,20288.0,20289.0,20290.0,20291.0,20292.0,20293.0,4.0,8.0,13.0,8.0,9.0,upward
2025-05-06 22:48:26.268,12.0,20343.0,10.0,7.0,13.0,9.0,5.0,20333.0,20334.0,20335.0,20336.0,20337.0,20337.5,20338.0,20339.0,20340.0,20341.0,20342.0,2.0,2.0,9.0,8.0,11.0,upward


In [11]:
import pandas as pd

def group_trade_through_events(df, time_col='time', label_col='trade_through', window_seconds=1):
    """
    將穿價成交依時間聚類：1 秒內視為同一事件
    會加入欄位：'time_diff', 'event_group'
    """
    df = df.sort_values(time_col).copy()
    # df[time_col] = pd.to_datetime(df[time_col]) 
    df['time_diff'] = df[time_col].diff().dt.total_seconds().fillna(9999)
    df['event_group'] = (df['time_diff'] > window_seconds).cumsum()
    return df


def summarize_trade_through_events(trade_through_df, window_seconds=1):
    """
    對穿價資料分群，輸出每組穿價事件的代表方向與時間
    """
    # 1. 還原 index 為欄位
    if isinstance(trade_through_df.index, pd.DatetimeIndex):
        trade_through_df = trade_through_df.reset_index()  # index: time → 欄位: time

    # 2. 分群
    grouped_df = group_trade_through_events(trade_through_df, time_col='time', window_seconds=window_seconds)

    # 3. 每組只保留第一筆穿價事件
    summary = grouped_df.groupby('event_group').first()[['time', 'trade_through']].reset_index()

    return summary


In [12]:
summary_df = summarize_trade_through_events(trade_through_df, window_seconds=1)

In [114]:
summary_df.index

DatetimeIndex(['2025-05-01 01:44:24.460000', '2025-05-01 02:07:04.506000',
               '2025-05-01 02:07:07.152000', '2025-05-01 02:29:33.142000',
               '2025-05-01 04:06:21.204000', '2025-05-01 04:07:07.237000',
               '2025-05-01 04:50:36.721000', '2025-05-01 04:59:02.356000',
               '2025-05-02 08:45:00.055000', '2025-05-02 08:45:03.325000',
               ...
               '2025-05-06 16:43:00.504000', '2025-05-06 19:23:12.617000',
               '2025-05-06 19:23:17.396000', '2025-05-06 21:04:06.093000',
               '2025-05-06 21:39:49.835000', '2025-05-06 22:20:17.656000',
               '2025-05-06 22:25:50.115000', '2025-05-06 22:36:59.694000',
               '2025-05-06 22:48:26.268000', '2025-05-06 23:31:12.200000'],
              dtype='datetime64[ns]', name='time', length=1770, freq=None)

### 市場TXF所有穿價的時間點及方向

### 獲取我們的成交資料(202505)

In [81]:
import sys
import os
sys.path.append(os.path.abspath(".."))
from eddie_tools.record import RedisFillLoader

In [82]:
tmf = RedisFillLoader('2025,5,1','2025,5,8','capital_electron_tmf')

In [83]:
df = tmf.load_concat_df()

Concatenating fills: 100%|██████████| 8/8 [00:00<00:00, 16.60it/s]


In [97]:
fills_df = df[df['instr'].str.contains('tmf:202505', case=False, na=False)]

> 把我們的成交紀錄與市場穿價紀錄做比對 並合併

In [98]:
def merge_and_group_fills(my_fills_df, summary_df, threshold_seconds=1):
    import pandas as pd

    # 確保按時間排序
    my_fills_df = my_fills_df.sort_values('time')
    summary_df = summary_df.sort_values('time')

    # merge_asof 以找出最接近穿價事件
    merged = pd.merge_asof(
        my_fills_df,
        summary_df,
        on='time',
        direction='backward',
        tolerance=pd.Timedelta(seconds=threshold_seconds)
    )
    merged['is_trade_near_cross'] = merged['trade_through'].notna()

    # 精簡欄位
    merged_df = merged[['time', 'px', 'sz', 'instr', 'trade_through', 'is_trade_near_cross']]

    # 群組內合併 (以秒為單位)
    def group_within_seconds(df, time_col='time', threshold_seconds=1):
        df = df.sort_values(time_col).copy()
        df[time_col] = pd.to_datetime(df[time_col])  # 確保時間格式
        df['time_diff'] = df[time_col].diff().dt.total_seconds().fillna(float('inf'))
        df['group_id'] = (df['time_diff'] > threshold_seconds).cumsum()
        return df

    grouped = group_within_seconds(merged_df, time_col='time', threshold_seconds=threshold_seconds)

    # 聚合每一群
    merged_sec_df = (
        grouped
        .groupby('group_id')
        .agg(
            time=('time', 'first'),
            sz=('sz', 'sum'),
            trade_through=('trade_through', 'first'),
            instr=('instr', 'first')
        )
        .reset_index(drop=True)
    )

    return merged_sec_df


### 使用成交量加權

In [99]:
def analyze_directional_fill_ratio(merged_df):
    df = merged_df.copy()

    # 上穿時成交的總 long / short 量
    upward_df = df[df['trade_through'] == 'upward']
    if not upward_df.empty:
        long_sz = upward_df[upward_df['sz'] > 0]['sz'].sum()
        short_sz = -upward_df[upward_df['sz'] < 0]['sz'].sum()  # 注意是負數要取負號
        total = long_sz + short_sz

        long_ratio = long_sz / total if total != 0 else 0
        short_ratio = short_sz / total if total != 0 else 0

        print(f"TXF上穿時 TMF long 量比例：{long_ratio:.2%}")
        print(f"TXF上穿時 TMF short 量比例：{short_ratio:.2%}")
    else:
        print("無 upward cross 資料")

    # 下穿時成交的總 long / short 量
    downward_df = df[df['trade_through'] == 'downward']
    if not downward_df.empty:
        long_sz = downward_df[downward_df['sz'] > 0]['sz'].sum()
        short_sz = -downward_df[downward_df['sz'] < 0]['sz'].sum()
        total = long_sz + short_sz

        long_ratio = long_sz / total if total != 0 else 0
        short_ratio = short_sz / total if total != 0 else 0

        print(f"TXF下穿時 TMF long 量比例：{long_ratio:.2%}")
        print(f"TXF下穿時 TMF short 量比例：{short_ratio:.2%}")
    else:
        print("無 downward cross 資料")

    return None


* 觀察Long(Short)TMF時 市場是否有穿價 方向又為何

In [101]:
def analyze_ratio_count_and_volume(df):
    df = df.copy()
    df['trade_through'] = df['trade_through'].fillna('no-through')

    for label, sub_df in [('long', df[df['sz'] > 0]), ('short', df[df['sz'] < 0])]:
        total_count = len(sub_df)
        total_volume = sub_df['sz'].abs().sum()

        if total_count == 0 or total_volume == 0:
            print(f"\n沒有 {label.upper()} TMF")
            continue

        ratio_count = sub_df['trade_through'].value_counts(normalize=True)
        ratio_volume = sub_df.groupby('trade_through')['sz'].apply(lambda x: x.abs().sum()) / total_volume

        print(f"\nElectron {label.upper()} TMF 時 :")
        print(f"只看成交-> TXF 價格上穿: {ratio_count.get('upward', 0):.2%}, TXF 價格下穿: {ratio_count.get('downward', 0):.2%}, 無穿價: {ratio_count.get('no-through', 0):.2%}")
        print(f"使用成交量加權-> TXF 價格上穿: {ratio_volume.get('upward', 0):.2%}, TXF 價格下穿: {ratio_volume.get('downward', 0):.2%}, 無穿價: {ratio_volume.get('no-through', 0):.2%}")


In [118]:
# merge_sec_df = merge_and_group_fills(fills_df, trade_through_df, threshold_seconds=1)
trade_through_df.columns

Index(['sz', 'px', 'bid_sz4', 'bid_sz3', 'bid_sz2', 'bid_sz1', 'bid_sz',
       'bid_px4', 'bid_px3', 'bid_px2', 'bid_px1', 'bid_px', 'mid_px',
       'ask_px', 'ask_px1', 'ask_px2', 'ask_px3', 'ask_px4', 'ask_sz',
       'ask_sz1', 'ask_sz2', 'ask_sz3', 'ask_sz4', 'trade_through'],
      dtype='object')

In [109]:
analyze_ratio_count_and_volume(merge_sec_df)


Electron LONG TMF 時 :
只看成交-> TXF 價格上穿: 10.53%, TXF 價格下穿: 8.42%, 無穿價: 81.05%
使用成交量加權-> TXF 價格上穿: 16.22%, TXF 價格下穿: 8.15%, 無穿價: 75.63%

Electron SHORT TMF 時 :
只看成交-> TXF 價格上穿: 3.03%, TXF 價格下穿: 16.67%, 無穿價: 80.30%
使用成交量加權-> TXF 價格上穿: 3.01%, TXF 價格下穿: 35.63%, 無穿價: 61.36%


* 看TMF有成交的時候 前一秒內有無發生穿價事件 為了避免重複計算 把同一秒的TMF成交合併 看那個時間點是偏long還是short 再往前查看是否有穿價以及穿價的方向

In [110]:
night_fills_df = fills_df.between_time('15:00:00','05:00:00')

TypeError: Index must be DatetimeIndex

In [70]:
summary_df = summary_df.set_index('time')

In [72]:
night_summary_df = summary_df.between_time('15:00:00','05:00:00')

In [74]:
my_night_fills_df = night_fills_df.sort_values('time')
night_summary_df = night_summary_df.sort_values('time')

# 往前merge
merged = pd.merge_asof(
    my_night_fills_df,
    night_summary_df,
    on='time',
    direction='backward',
    tolerance=pd.Timedelta(seconds=1)  
)


In [75]:
merged['is_trade_near_cross'] = merged['trade_through'].notna()
merged_df = merged[['time','px','sz','instr','trade_through','is_trade_near_cross']]

In [76]:

grouped = group_within_seconds(merged_df, time_col='time', threshold_seconds=1)

# 合併同一群內的 sz，其餘欄位依需要處理
merged_sec_df = (
    grouped
    .groupby('group_id')
    .agg(
        time=('time', 'first'),                  # 群組代表時間取第一筆
        sz=('sz', 'sum'),                        # 口數相加
        trade_through=('trade_through', 'first'),# 穿價方向取第一筆
        instr=('instr', 'first') if 'instr' in merged_df.columns else ('dummy', 'first')
    )
    .reset_index(drop=True)
)


In [77]:
analyze_ratio_count_and_volume(merged_sec_df)


Electron LONG TMF 時 :
只看成交-> TXF 價格上穿: 8.97%, TXF 價格下穿: 10.26%, 無穿價: 80.77%
使用成交量加權-> TXF 價格上穿: 14.33%, TXF 價格下穿: 9.10%, 無穿價: 76.57%

Electron SHORT TMF 時 :
只看成交-> TXF 價格上穿: 3.85%, TXF 價格下穿: 21.15%, 無穿價: 75.00%
使用成交量加權-> TXF 價格上穿: 3.57%, TXF 價格下穿: 42.20%, 無穿價: 54.23%


In [78]:
def merge_and_group_fills(my_fills_df, summary_df, threshold_seconds=1):
    import pandas as pd

    # 確保按時間排序
    my_fills_df = my_fills_df.sort_values('time')
    summary_df = summary_df.sort_values('time')

    # merge_asof 以找出最接近穿價事件
    merged = pd.merge_asof(
        my_fills_df,
        summary_df,
        on='time',
        direction='backward',
        tolerance=pd.Timedelta(seconds=threshold_seconds)
    )
    merged['is_trade_near_cross'] = merged['trade_through'].notna()

    # 精簡欄位
    merged_df = merged[['time', 'px', 'sz', 'instr', 'trade_through', 'is_trade_near_cross']]

    # 群組內合併 (以秒為單位)
    def group_within_seconds(df, time_col='time', threshold_seconds=1):
        df = df.sort_values(time_col).copy()
        df[time_col] = pd.to_datetime(df[time_col])  # 確保時間格式
        df['time_diff'] = df[time_col].diff().dt.total_seconds().fillna(float('inf'))
        df['group_id'] = (df['time_diff'] > threshold_seconds).cumsum()
        return df

    grouped = group_within_seconds(merged_df, time_col='time', threshold_seconds=threshold_seconds)

    # 聚合每一群
    merged_sec_df = (
        grouped
        .groupby('group_id')
        .agg(
            time=('time', 'first'),
            sz=('sz', 'sum'),
            trade_through=('trade_through', 'first'),
            instr=('instr', 'first')
        )
        .reset_index(drop=True)
    )

    return merged_sec_df


In [79]:
merged_sec_df = merge_and_group_fills(my_fills_df, summary_df, threshold_seconds=1)


In [103]:
analyze_ratio_count_and_volume(merged_sec_df)


Electron LONG TMF 時 :
只看成交-> TXF 價格上穿: 10.53%, TXF 價格下穿: 8.42%, 無穿價: 81.05%
使用成交量加權-> TXF 價格上穿: 16.22%, TXF 價格下穿: 8.15%, 無穿價: 75.63%

Electron SHORT TMF 時 :
只看成交-> TXF 價格上穿: 3.03%, TXF 價格下穿: 16.67%, 無穿價: 80.30%
使用成交量加權-> TXF 價格上穿: 3.01%, TXF 價格下穿: 35.63%, 無穿價: 61.36%


In [130]:
def group_fills_within_seconds(fills_df, threshold_seconds=1):
    df = fills_df.reset_index().sort_values('time').copy()  # <-- 這裡加上 reset_index()
    df['time'] = pd.to_datetime(df['time'])
    df['time_diff'] = df['time'].diff().dt.total_seconds().fillna(float('inf'))
    df['group_id'] = (df['time_diff'] > threshold_seconds).cumsum()

    grouped = (
        df.groupby('group_id')
        .agg(
            time=('time', 'first'),
            sz=('sz', 'sum'),
            px=('px', 'mean'),
            instr=('instr', 'first')
        )
        .reset_index(drop=True)
    )

    return grouped


In [ ]:
def match_grouped_fills_to_trade_through(grouped_fills_df, trade_through_df, threshold_seconds=1):
    grouped_fills_df = grouped_fills_df.sort_values("time").copy()
    trade_through_df = trade_through_df.sort_values("time").copy()
    trade_through_df = trade_through_df[["time", "trade_through"]]  # 精簡欄位

    merged = pd.merge_asof(
        grouped_fills_df,
        trade_through_df,
        on="time",
        direction="backward",
        tolerance=pd.Timedelta(seconds=threshold_seconds)
    )
    merged["is_trade_near_cross"] = merged["trade_through"].notna()

    return merged


In [137]:
grouped_fills = group_fills_within_seconds(fills_df, threshold_seconds=1)
merged_sec_df = match_grouped_fills_to_trade_through(grouped_fills, trade_through_df, threshold_seconds=1)


In [138]:
merged_sec_df

,time,sz,px,instr,trade_through,is_trade_near_cross
0,2025-04-30 15:00:05.508137,5,20184.333333,FUT_TAIFEX_TMF:202505,NaN,False
1,2025-04-30 15:00:09.305926,-5,20186.000000,FUT_TAIFEX_TMF:202505,NaN,False
2,2025-04-30 19:28:14.002964,-2,20211.000000,FUT_TAIFEX_TMF:202505,NaN,False
3,2025-04-30 20:46:53.155648,-40,20044.000000,FUT_TAIFEX_TMF:202505,NaN,False
4,2025-04-30 20:53:04.288506,2,20028.000000,FUT_TAIFEX_TMF:202505,NaN,False
...,...,...,...,...,...,...
160,2025-05-08 08:45:00.740773,-1,20487.000000,FUT_TAIFEX_TMF:202505,NaN,False
161,2025-05-08 08:45:02.501321,5,20485.000000,FUT_TAIFEX_TMF:202505,NaN,False
162,2025-05-08 11:34:41.492768,18,20665.000000,FUT_TAIFEX_TMF:202505,NaN,False
163,2025-05-08 11:34:58.520004,2,20669.000000,FUT_TAIFEX_TMF:202505,NaN,False


In [139]:
analyze_ratio_count_and_volume(merged_sec_df)


Electron LONG TMF 時 :
只看成交-> TXF 價格上穿: 8.42%, TXF 價格下穿: 8.42%, 無穿價: 83.16%
使用成交量加權-> TXF 價格上穿: 5.87%, TXF 價格下穿: 8.15%, 無穿價: 85.98%

Electron SHORT TMF 時 :
只看成交-> TXF 價格上穿: 3.03%, TXF 價格下穿: 16.67%, 無穿價: 80.30%
使用成交量加權-> TXF 價格上穿: 3.01%, TXF 價格下穿: 35.63%, 無穿價: 61.36%
